<a href="https://colab.research.google.com/github/MarianoNaveyra/DataScience/blob/main/Proyecto_Partell_Naveyra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [7]:
url1 = 'https://raw.githubusercontent.com/MarianoNaveyra/DataScience/main/Dataset.csv'

df1 = pd.read_csv(url1 , delimiter=';')

df1.head()

,Ruteo,Doc.,Plan,Fe.entrega,Material,Q entregada,Camion,Familia,Grupo,Subgrupo,...,B. PKN,Kg. PICKING,Kg. TOTAL,UC Tot,Acumulado,Ticket,DTT + TICKET,Posicion,Tipo Preparacion,Grupo acumulado
0,1,5509787,OR00062089,07.08.2024,100434,1,3001,1,1,1,...,1,"5,15","5,15","0,880591758","0,1",1,5509787 - 1,1,1,1
1,1,5509787,OR00062089,07.08.2024,102913,20,3001,1,9,5,...,20,192,192,"31,70130328","0,81",1,5509787 - 1,2,1,1
2,1,5509787,OR00062089,07.08.2024,100413,5,3001,1,9,5,...,5,48,48,"7,925325819","1,04",1,5509787 - 1,3,1,1
3,1,5509787,OR00062089,07.08.2024,103113,2,3001,1,9,5,...,2,"19,2","19,2","3,170130328","1,13",1,5509787 - 1,4,1,1
4,1,5509787,OR00062089,07.08.2024,100820,2,3001,1,9,5,...,2,"19,2","19,2","3,170130328","1,22",1,5509787 - 1,5,1,1


In [9]:
url2 = 'https://raw.githubusercontent.com/MarianoNaveyra/DataScience/main/fake_customer_data.csv'

df2 = pd.read_csv(url2)

df2.head()

,CustomerID,Age,Gender,City,State,Annual_Income,Purchase_History,Visit_Frequency,Average_Session_Duration,Last_Purchase_Date,Items_in_Cart,Payment_Method,Purchase_Amount,Discount_Used,Customer_Satisfaction
0,c174b6ad-4f13-4836-a221-9d9928b4d7be,60,Other,Lake Austin,North Dakota,113963.34,Home,13,24.61,2023-12-01,5,PayPal,317.57,No,10
1,f9cf612b-a8fd-447e-bebe-1c40e98dbe35,65,Female,Port Cynthiaberg,Mississippi,74053.27,Home,5,42.58,2023-09-26,3,Credit Card,195.32,Yes,4
2,c2195b47-c748-435c-8f9c-35b75b99001d,35,Female,East Tiffany,Missouri,106887.75,Home,10,47.41,2024-01-24,5,Debit Card,252.46,No,3
3,2190a9ad-5a9d-4530-8a50-d6c30c31c821,45,Other,Lake John,Idaho,141842.24,Electronics,2,31.11,2023-05-07,9,Credit Card,58.83,Yes,9
4,321a4609-6b82-4bd7-8b29-a29031899ca9,59,Other,Lake Jennifer,Missouri,125506.38,Fashion,4,14.51,2023-10-05,5,PayPal,307.28,No,3


In [10]:
url3 = 'https://raw.githubusercontent.com/MarianoNaveyra/DataScience/main/football_data.csv'

df3 = pd.read_csv(url3)

df3.head()

,Team,Opponent,Match_Type,Date,Location,Team_Score,Opponent_Score,Possession_Team,Possession_Opponent,Shots_Team,Shots_Opponent,Fouls_Team,Fouls_Opponent,Yellow_Cards_Team,Yellow_Cards_Opponent,Red_Cards_Team,Red_Cards_Opponent
0,England,Germany,Tournament,2010-01-01,Away,3,4,39.910873,47.305298,29,18,9,9,1,4,1,0
1,Germany,France,Qualifier,2010-01-02,Away,3,0,59.927642,62.764146,23,20,16,8,3,2,1,0
2,Portugal,Belgium,Tournament,2010-01-03,Neutral,1,4,54.083655,57.671462,9,28,15,18,3,2,1,1
3,Spain,Spain,Friendly,2010-01-04,Neutral,0,4,52.601094,34.814670,16,14,13,13,1,3,0,0
4,England,Spain,Qualifier,2010-01-05,Home,0,1,35.972048,51.594917,26,12,11,6,3,1,1,0
